In [1]:
import pandas as pd
import bs4
import glob
import re
import sys
import tqdm

from ferment_wine import ferment_wine

In [2]:
DATA_DIR = 'E:/wine_pages/'

In [3]:
scraping_log = pd.read_csv(DATA_DIR+'log.txt', header=None, sep='\t')
scraping_log['response_code'] = scraping_log[1].str.split('<ResponseCode>:', expand=True)[1]
scraping_log['url'] = scraping_log[0].str.split('<URL>:', expand=True)[1]
scraping_log = scraping_log[['url','response_code']]

scraping_log.head()

,url,response_code
0,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,200
1,/product/Louis-Roederer-Cristal-Brut-with-Two-...,200
2,/product/Laurent-Perrier-Cuvee-Rose/10521,200
3,/product/Piper-Heidsieck-Cuvee-Brut-in-Travel-...,200
4,/product/Clarendon-Hills-Astralis-Syrah-2011/5...,200


In [4]:
scraping_log['response_code'].value_counts()

200    130218
404        80
503        43
502         6
500         1
Name: response_code, dtype: int64

In [17]:
file_paths = glob.glob(DATA_DIR+'200*html')
print(f"Total 200 status pages found: {len(file_paths):,}")

Total 200 status pages found: 125,950


In [18]:
import multiprocessing as mp

p = mp.Pool(processes=16)
df_list = p.map(ferment_wine, file_paths)

In [ ]:
errs = wine_df[wine_df['category_1'].isnull()]
print(errs.shape)

In [23]:
wine_df.isnull().sum()

name               0
description        0
price              0
category_1         0
category_2         0
origin             0
wine_size_value    0
wine_size_units    0
wine_abv           0
dtype: int64

In [ ]:
wine_df.sample(50)

In [22]:
wine_df = pd.concat(df_list)
print(wine_df.shape)

wine_df.to_pickle('data/best_wine_df_ever.pickle.gzip', compression='gzip')

(123516, 9)


# Scratch

In [14]:
print(bad_wines[0])

('E:/wine_pages\\200_∕product∕1000-Stories-Bourbon-Barrel-Aged-Batch-Blue-Carignan-2016∕431257.html', 'Traceback (most recent call last):\n  File "C:\\Users\\david\\Documents\\github\\this-wine-does-not-exist\\ferment_wine.py", line 63, in ferment_wine\n    wine_df.append(wine_row)\nNameError: name \'wine_df\' is not defined\n')


In [139]:
wine_name = soup.find('h1',attrs={'itemprop':'name'}).text
wine_name

'90+ Point Cabernet & Godiva Limited Edition Holiday Chocolates'

In [133]:
wine_description = soup.find('div', attrs={'class':'viewMoreModule_text'})
print(wine_description)
if len(wine_description.find_all('p')) > 0:
  fixed_description = '\n'.join(str(tag.text) for tag in wine_description.find_all('p'))
else:
  fixed_description = wine_description.text
print("\n",fixed_description)

<div class="viewMoreModule_text"><p>Irresistible all over, the rich and fruity Baileyana Firepeak Pinot Noir is paired with this holiday designed gift box from Godiva that is filled with an exceptional assortment of truffles including fabulous flavors, such as Salted Caramel, Roasted Almond, Christmas Cookie, Cranberry, and more.</p><br/><b>Wine Gift Includes:</b><ul><li><b>Baileyana Firepeak Pinot Noir</b><br/>Aromas of bright cherry, blackberry, rose petal, and hints of spice burst from the glass of this bright and nicely balanced Pinot Noir. Flavors of dark fruit, blueberry, mocha, and cola flavors along with soft supple tannins create a beautiful finish.</li><li><b>Godiva Truffles Seasonal Gift Box, 12 pc.</b><br/>Godiva's gorgeous green Godiva Holiday Truffles Gift Box boasts a beautiful Christmas tree illustration created by Swedish-born artist and designer Lotta Kühlhorn. Inside, the chocolate assortment includes 12 delectable Belgian chocolate truffles in fabulous flavors, such

In [130]:
wine_price = soup.find('div', attrs={'class','productPrice'})
if wine_price is None:
  wine_price = soup.find('meta', attrs={'itemprop':'price'})['content']
else:
  wine_price = wine_price.find('meta', {'itemprop':'price'})['content']
print(wine_price)

18.99


In [129]:
soup.find('meta', attrs={'itemprop':'price'})['content']

'18.99'

In [125]:
soup.find('div', attrs={'class','productPrice'})

In [100]:
wine_category_1 = soup.find("ul", attrs={"class":"prodAttr"}).find("li")['title']
print(wine_category_1)

White Wine


In [101]:
wine_category_2 = soup.find("span", {"class":"prodItemInfo_varietal"}).text
print(wine_category_2)

Pinot Gris/Grigio


In [102]:
wine_origin = soup.find("span", {"class":"prodItemInfo_originText"}).text
print(wine_origin)

 from Veneto, Italy


In [103]:
wine_size_value = soup.find("span", attrs={"class":"prodAlcoholVolume_text"}).text
print(wine_size_value)

750


In [106]:
wine_size_units = re.findall("[a-zA-Z]+", soup.find("span", attrs={"class":"prodAlcoholVolume"}).text)[0]
print(wine_size_units)

ML


In [107]:
wine_abv = soup.find("span", attrs={"class":"prodAlcoholPercent_percent"}).text
print(wine_abv)

12.5


---------------------

# OLD

In [4]:
raw_df = pd.read_pickle('data/scraped/scraped_with_decs.pickle.gzip', compression='gzip')
print(raw_df.shape)

(125787, 7)


In [102]:
raw_df.head(3)

,name,image_path,price,url,raw_html,URL_name,description
0,Dom Perignon Lenny Kravitz Limited Edition wit...,/product/images/fl_progressive/lmgmud1xsenlouw...,199.97,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,Dom Pérignon Vintage 2008 has been distinguish...
1,Louis Roederer Cristal Brut with Two Flutes an...,/product/images/fl_progressive/iqec7e0nf6mzyof...,329.99,/product/Louis-Roederer-Cristal-Brut-with-Two-...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Louis-Roederer-Cristal-Brut-with-Two-...,Gift pack containing one bottle of 2008 Louis ...
2,Laurent-Perrier Cuvee Rose,/product/images/fl_progressive/10521.jpg,79.99,/product/Laurent-Perrier-Cuvee-Rose/10521,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Laurent-Perrier-Cuvee-Rose/10521,The Cuvée Rosé from Laurent-Perrier is the mos...


In [7]:
master_df = raw_df.copy()

In [36]:
for ix,row in master_df.iterrows():
  raw_html = row['raw_html']
  soup = bs4.BeautifulSoup(raw_html)
  if ix == 2:
    break

In [63]:
wine_category_1 = soup.find("ul", attrs={"class":"prodAttr"}).find("li")['title']
print(wine_category_1)

Sparkling & Champagne


In [64]:
wine_category_2 = soup.find("span", {"class":"prodItemInfo_varietal"}).text
print(wine_category_2)

Rosé Sparkling Wine


In [65]:
wine_origin = soup.find("span", {"class":"prodItemInfo_originText"}).text
print(wine_origin)

Champagne, France


In [118]:
wine_size_value = soup.find("span", attrs={"class":"prodAlcoholVolume_text"}).text
print(wine_size_value)

750


In [123]:
wine_size_units = re.findall("[a-zA-Z]+", soup.find("span", attrs={"class":"prodAlcoholVolume"}).text)[0]
print(wine_size_units)

ML


In [125]:
wine_abv = soup.find("span", attrs={"class":"prodAlcoholPercent_percent"}).text
print(wine_abv)

12


In [93]:
soup.find("span", attrs={"class":"prodAlcoholPercent_percent"}).text


<div class="prodAlcoholPercent" data-view1_32="true">
<div class="prodAlcoholPercent_content">
<span class="prodAlcoholVolume">
<span class="prodAlcoholVolume_text">750</span>ML /
        </span>
<span class="prodAlcoholPercent_inner">
<span class="prodAlcoholPercent_percent">12</span>% ABV
        </span>
</div>
</div>

In [117]:
import re

re.findall("[a-zA-Z]+", soup.find("span", attrs={"class":"prodAlcoholVolume"}).text)

['ML']

In [110]:
soup.find("span", attrs={"class":"prodAlcoholVolume_text"}).text

'750'

In [108]:
soup.find("span", attrs={"class":"prodAlcoholVolume"}).text.strip()

'750ML /'

In [94]:
url = "https://www.wine.com/product/laurent-perrier-cuvee-rose/10521"
html = requests.get(url)

In [101]:
str(html)

'<Response [200]>'

In [82]:
import requests

url = "https://www.wine.com/product/laurent-perrier-cuvee-rose/10521"
html = requests.get(url).content

soup = bs4.BeautifulSoup(html, "html.parser")

In [83]:
soup.find("div", attrs={"class":"prodAlcoholPercent"})

<div class="prodAlcoholPercent" data-view1_32="true">
<div class="prodAlcoholPercent_content">
<span class="prodAlcoholVolume">
<span class="prodAlcoholVolume_text">750</span>ML /
        </span>
<span class="prodAlcoholPercent_inner">
<span class="prodAlcoholPercent_percent">12</span>% ABV
        </span>
</div>
</div>

In [85]:
old_scrape = pd.read_pickle('data/scraped/scrape.pickle.gz', compression='gzip')
old_scrape.head(4)

,name,image_path,price,url,raw_html
0,Dom Perignon Lenny Kravitz Limited Edition wit...,/product/images/fl_progressive/lmgmud1xsenlouw...,199.97,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
1,Louis Roederer Cristal Brut with Two Flutes an...,/product/images/fl_progressive/iqec7e0nf6mzyof...,329.99,/product/Louis-Roederer-Cristal-Brut-with-Two-...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
2,Laurent-Perrier Cuvee Rose,/product/images/fl_progressive/10521.jpg,79.99,/product/Laurent-Perrier-Cuvee-Rose/10521,"<div class=""prodItem_wrap"">\n<div class=""prodI..."
3,Piper-Heidsieck Cuvee Brut in Travel Case with...,/product/images/fl_progressive/aqi87aqobd3zc56...,79.99,/product/Piper-Heidsieck-Cuvee-Brut-in-Travel-...,"<div class=""prodItem_wrap"">\n<div class=""prodI..."


In [92]:
old_scrape.iloc[2,:]['url']

'/product/Laurent-Perrier-Cuvee-Rose/10521'

In [91]:
bs4.BeautifulSoup(old_scrape.iloc[2,:]['raw_html'])

<html><body><div class="prodItem_wrap">
<div class="prodItemImage" data-view1_19_1_1_2="true">
<a class="prodItemImage_link" href="/product/Laurent-Perrier-Cuvee-Rose/10521" tabindex="-1"><picture class="prodItemImage_image" style="background-image: url(/product/images/w_160,h_160,c_fit,q_auto:good,fl_progressive/10521.jpg)"><source class="prodItemImage_image-phone" media="(max-width: 767px)" srcset="/product/images/w_105,h_105,c_fit,q_auto:good,fl_progressive/10521.jpg 1x, /product/images/w_105,h_105,dpr_2.0,c_fit,q_auto:good,fl_progressive/10521.jpg 2x"><source class="prodItemImage_image-tablet" media="(min-width: 768px)" srcset="/product/images/w_160,h_160,c_fit,q_auto:good,fl_progressive/10521.jpg 1x, /product/images/w_160,h_160,dpr_2.0,c_fit,q_auto:good,fl_progressive/10521.jpg 2x"><img alt="Laurent-Perrier Cuvee Rose Front Label" class="prodItemImage_image-default" itemprop="image" src="/product/images/fl_progressive/10521.jpg"/></source></source></picture></a>
</div><div class="

In [69]:
soup

<html><body><div class="prodItem_wrap">
<div class="prodItemImage" data-view1_19_1_1_2="true">
<a class="prodItemImage_link" href="/product/Laurent-Perrier-Cuvee-Rose/10521" tabindex="-1"><picture class="prodItemImage_image" style="background-image: url(/product/images/w_160,h_160,c_fit,q_auto:good,fl_progressive/10521.jpg)"><source class="prodItemImage_image-phone" media="(max-width: 767px)" srcset="/product/images/w_105,h_105,c_fit,q_auto:good,fl_progressive/10521.jpg 1x, /product/images/w_105,h_105,dpr_2.0,c_fit,q_auto:good,fl_progressive/10521.jpg 2x"><source class="prodItemImage_image-tablet" media="(min-width: 768px)" srcset="/product/images/w_160,h_160,c_fit,q_auto:good,fl_progressive/10521.jpg 1x, /product/images/w_160,h_160,dpr_2.0,c_fit,q_auto:good,fl_progressive/10521.jpg 2x"><img alt="Laurent-Perrier Cuvee Rose Front Label" class="prodItemImage_image-default" itemprop="image" src="/product/images/fl_progressive/10521.jpg"/></source></source></picture></a>
</div><div class="